# Model 2: Data Augmentation

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 32
n_classes = 20
n_epochs = 15
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - Single convolution layer and dense layer

In [6]:
# defining the model architecture
model_02 = models.Sequential()

# convolution/max pool stacks
model_02.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
model_02.add(layers.MaxPooling2D((2,2)))

model_02.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
model_02.add(layers.MaxPooling2D((2,2)))

model_02.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
model_02.add(layers.MaxPooling2D((2,2)))

model_02.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
model_02.add(layers.MaxPooling2D((2,2)))

# fully connected layers
model_02.add(layers.Flatten())
model_02.add(layers.Dense(512, activation='relu'))
model_02.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
model_02.summary()
model_names.append('model_02')
model_list.append(model_02)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 256)      

## Step 4: Training

In [7]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 341 ms, sys: 936 ms, total: 1.28 s
Wall time: 1.28 s


In [9]:
# compiling loss functions
model_02.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [10]:
%%time
# setting up model saving checkpoints
model_02_cp = ModelCheckpoint(filepath=f'{model_path}model_02.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
model_02_history = model_02.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[model_02_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(model_02_history)

Epoch 1/15
253/253 [==============================] - 136s 537ms/step - loss: 2.8623 - acc: 0.1026 - val_loss: 2.5536 - val_acc: 0.1750

Epoch 00001: val_acc improved from -inf to 0.17500, saving model to /floyd/home/model_02.h5
Epoch 2/15
253/253 [==============================] - 71s 282ms/step - loss: 2.4822 - acc: 0.2011 - val_loss: 2.6131 - val_acc: 0.2635

Epoch 00002: val_acc improved from 0.17500 to 0.26350, saving model to /floyd/home/model_02.h5
Epoch 3/15
253/253 [==============================] - 71s 282ms/step - loss: 2.3062 - acc: 0.2488 - val_loss: 2.1474 - val_acc: 0.3072

Epoch 00003: val_acc improved from 0.26350 to 0.30720, saving model to /floyd/home/model_02.h5
Epoch 4/15
253/253 [==============================] - 71s 280ms/step - loss: 2.2130 - acc: 0.2784 - val_loss: 1.7737 - val_acc: 0.3483

Epoch 00004: val_acc improved from 0.30720 to 0.34833, saving model to /floyd/home/model_02.h5
Epoch 5/15
253/253 [==============================] - 71s 279ms/step - loss: 2

In [11]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [12]:
# evaluating models on validation set
for key, value in models_dict.items():
    model = models.load_model(f'{model_path}{key}.h5')
    (val_loss, val_accuracy) = model.evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

26/26 [==============================] - 1s 42ms/step
model_02 Val Accuracy: 47.78%
model_02 Val Loss: 2.2701
---


In [13]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

model_02_history saved in /floyd/home/
